## End to end model:Classifier

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras import optimizers
from keras.applications.vgg19 import preprocess_input
from keras.utils.training_utils import multi_gpu_model
from keras.utils import to_categorical
import tensorflow as tf
import cv2
import os
import random
import h5py
import numpy as np

Using TensorFlow backend.


## 1 read feature and label

In [2]:
h5_files = ['/home/tsimage/high_speed_data/end2end_feature/feature_label_500.h5',
            '/home/tsimage/high_speed_data/end2end_feature/feature_label_1000.h5',
            '/home/tsimage/high_speed_data/end2end_feature/feature_label_1500.h5',
            '/home/tsimage/high_speed_data/end2end_feature/feature_label_2000.h5']

for i, h5_file in enumerate(h5_files):
    h5f = h5py.File(h5_file,'r')
    features_h5 = h5f['features'][:]
    labels_h5 = h5f['labels'][:]
    h5f.close()
    if (i == 0):
        print("du ###",i)
        features_h5_all = features_h5
        labels_h5_all = labels_h5
        print("du", features_h5_all.shape)
        print("du", labels_h5_all.shape)
        print("du", features_h5.shape)
        print("du", labels_h5.shape)
        features_h5 = None
        labels_h5 = None
    else:
        print("du ***",i)
        print("du", features_h5_all.shape)
        print("du", labels_h5_all.shape)
        print("du", features_h5.shape)
        print("du", labels_h5.shape)
        features_h5_all = np.append(features_h5_all, features_h5, axis = 0)
        labels_h5_all = np.append(labels_h5_all, labels_h5,  axis = 0)
        features_h5 = None
        labels_h5 = None
        
print("du", features_h5_all.shape)
print("du", labels_h5_all.shape)

du ### 0
du (500, 128, 128, 512)
du (500,)
du (500, 128, 128, 512)
du (500,)
du *** 1
du (500, 128, 128, 512)
du (500,)
du (491, 128, 128, 512)
du (491,)
du *** 2
du (991, 128, 128, 512)
du (991,)
du (500, 128, 128, 512)
du (500,)
du *** 3
du (1491, 128, 128, 512)
du (1491,)
du (500, 128, 128, 512)
du (500,)
du (1991, 128, 128, 512)
du (1991,)


In [3]:
features = features_h5_all
labels = labels_h5_all.reshape([len(labels_h5_all),1])
features_h5_all = None
labels_h5_all = None

print("du", features.shape)
print("du", labels.shape)

labels = to_categorical(labels,2)
print("du", features.shape)
print("du", labels.shape)

index = [i for i in range(len(features))]  
random.shuffle(index)
features = features[index]
labels = labels[index]

print("du", features.shape)
print("du", labels.shape)

du (1991, 128, 128, 512)
du (1991, 1)
du (1991, 128, 128, 512)
du (1991, 2)
du (1991, 128, 128, 512)
du (1991, 2)


## 2 build classifier

In [ ]:
with tf.device('/cpu:0'):
    input_tensor = Input((128, 128, 512))
    
    conv1_1 = Conv2D(filters = 1024, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv1_1')(input_tensor)
    conv1_2 = Conv2D(filters = 1024, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv1_2')(conv1_1)
    mp1 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(conv1_2)
    
    conv2_1 = Conv2D(filters = 2048, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv2_1')(mp1)
    conv2_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv2_2')(conv2_1)
    mp2 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(conv2_2)
    
    conv3_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv3_1')(mp2)
    conv3_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv3_2')(conv3_1)
    mp3 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(conv3_2)
    
    conv4_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv4_1')(mp3)
    conv4_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv4_2')(conv4_1)
    mp4 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(conv4_2)
    
    conv5_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv5_1')(mp4)
    conv5_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv5_2')(conv5_1)
    
#    flatten = Flatten(name='flatten')(conv8)
#    fc1 = Dense(1024, activation='relu', name='fc1')(flatten)
#    gap = GlobalAveragePooling2D()(conv8)
    gmp = GlobalMaxPooling2D()(conv5_2)
    drop = Dropout(0.5, name='drop_fc1')(gmp)
    predictions = Dense(2, activation='softmax')(drop)

    model = Model(inputs=input_tensor, outputs=predictions)
    model.summary()
# LearningRate = LearningRate * 1/(1 + decay * epoch)
    sgd = optimizers.SGD(lr=0.001, decay=0.001 * 100, momentum=0.9, nesterov=True)

parallel_model = multi_gpu_model(model, gpus=4)
#parallel_model.compile(optimizer='Adadelta', 
#                       loss='categorical_crossentropy', 
#                       metrics=['accuracy'])
parallel_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
parallel_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 512)     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 128, 128, 1024)    4719616   
_________________________________________________________________
conv1_2 (Conv2D)             (None, 128, 128, 1024)    1049600   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 1024)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 64, 64, 2048)      18876416  
_________________________________________________________________
conv2_2 (Conv2D)             (None, 64, 64, 2048)      4196352   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 2048)      0         
__________

In [ ]:
print("du", features.shape)
print("du", labels.shape)
parallel_model.fit(x=features, y=labels, 
                   batch_size=16, epochs=100, verbose=1, validation_split=0.2)
model.save_weights("diagnosis.h5")

du (1991, 128, 128, 512)
du (1991, 2)
Train on 1592 samples, validate on 399 samples
Epoch 1/100
1592/1592 [==============================] - 255s 160ms/step - loss: 0.5386 - acc: 0.7657 - val_loss: 0.5531 - val_acc: 0.7343
Epoch 2/100
1584/1592 [============================>.] - ETA: 1s - loss: 0.5195 - acc: 0.7664

### use sgd
(1991, 128, 128, 512)
du (1991, 2)
Train on 1592 samples, validate on 399 samples
Epoch 1/100
1592/1592 [==============================] - 261s 164ms/step - loss: 0.5493 - acc: 0.7563 - val_loss: 0.5050 - val_acc: 0.7744
Epoch 2/100
1592/1592 [==============================] - 245s 154ms/step - loss: 0.5126 - acc: 0.7563 - val_loss: 0.4751 - val_acc: 0.7744
Epoch 3/100
1592/1592 [==============================] - 245s 154ms/step - loss: 0.4957 - acc: 0.7563 - val_loss: 0.4657 - val_acc: 0.7744
Epoch 4/100
1592/1592 [==============================] - 246s 154ms/step - loss: 0.4690 - acc: 0.7563 - val_loss: 0.5274 - val_acc: 0.8446
Epoch 5/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.4889 - acc: 0.7619 - val_loss: 0.4390 - val_acc: 0.7744
Epoch 6/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.4466 - acc: 0.8065 - val_loss: 0.4691 - val_acc: 0.7870
Epoch 7/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.4403 - acc: 0.8034 - val_loss: 0.4116 - val_acc: 0.8697
Epoch 8/100
1592/1592 [==============================] - 246s 155ms/step - loss: 0.4092 - acc: 0.8367 - val_loss: 0.4116 - val_acc: 0.8546
Epoch 9/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.4005 - acc: 0.8398 - val_loss: 0.3637 - val_acc: 0.8847
Epoch 10/100
1592/1592 [==============================] - 249s 156ms/step - loss: 0.3774 - acc: 0.8480 - val_loss: 0.3688 - val_acc: 0.8546
Epoch 11/100
1592/1592 [==============================] - 249s 156ms/step - loss: 0.3726 - acc: 0.8411 - val_loss: 0.5530 - val_acc: 0.7744
Epoch 12/100
1592/1592 [==============================] - 246s 155ms/step - loss: 0.3694 - acc: 0.8448 - val_loss: 0.4641 - val_acc: 0.7769
Epoch 13/100
1592/1592 [==============================] - 246s 155ms/step - loss: 0.3649 - acc: 0.8455 - val_loss: 0.3210 - val_acc: 0.8772
Epoch 14/100
1592/1592 [==============================] - 246s 155ms/step - loss: 0.3699 - acc: 0.8354 - val_loss: 0.3399 - val_acc: 0.8822
Epoch 15/100
1592/1592 [==============================] - 248s 156ms/step - loss: 0.3509 - acc: 0.8555 - val_loss: 0.3752 - val_acc: 0.8546
Epoch 16/100
1592/1592 [==============================] - 246s 154ms/step - loss: 0.3470 - acc: 0.8536 - val_loss: 0.3162 - val_acc: 0.8922
Epoch 17/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.3344 - acc: 0.8574 - val_loss: 0.3222 - val_acc: 0.8797
Epoch 18/100
1592/1592 [==============================] - 248s 156ms/step - loss: 0.3352 - acc: 0.8562 - val_loss: 0.3446 - val_acc: 0.8747
Epoch 19/100
1592/1592 [==============================] - 248s 156ms/step - loss: 0.3191 - acc: 0.8624 - val_loss: 0.3349 - val_acc: 0.8772
Epoch 20/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.3233 - acc: 0.8543 - val_loss: 0.3266 - val_acc: 0.8772
Epoch 21/100
1592/1592 [==============================] - 249s 156ms/step - loss: 0.3293 - acc: 0.8555 - val_loss: 0.3073 - val_acc: 0.8747
Epoch 22/100
1592/1592 [==============================] - 245s 154ms/step - loss: 0.3079 - acc: 0.8693 - val_loss: 0.2785 - val_acc: 0.8872
Epoch 23/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2889 - acc: 0.8763 - val_loss: 0.3887 - val_acc: 0.8095
Epoch 24/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2971 - acc: 0.8599 - val_loss: 0.3806 - val_acc: 0.8571
Epoch 25/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2902 - acc: 0.8737 - val_loss: 0.2491 - val_acc: 0.8997
Epoch 26/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.3252 - acc: 0.8668 - val_loss: 0.2662 - val_acc: 0.8947
Epoch 27/100
1592/1592 [==============================] - 248s 156ms/step - loss: 0.2824 - acc: 0.8825 - val_loss: 0.3140 - val_acc: 0.8747
Epoch 28/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2704 - acc: 0.8844 - val_loss: 0.2405 - val_acc: 0.9048
Epoch 29/100
1592/1592 [==============================] - 248s 156ms/step - loss: 0.2895 - acc: 0.8706 - val_loss: 0.2520 - val_acc: 0.8922
Epoch 30/100
1592/1592 [==============================] - 249s 157ms/step - loss: 0.2849 - acc: 0.8744 - val_loss: 0.2430 - val_acc: 0.9098
Epoch 31/100
1592/1592 [==============================] - 248s 156ms/step - loss: 0.2569 - acc: 0.8781 - val_loss: 0.3845 - val_acc: 0.8797
Epoch 32/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2650 - acc: 0.8781 - val_loss: 0.2286 - val_acc: 0.9173
Epoch 33/100
1592/1592 [==============================] - 246s 155ms/step - loss: 0.2624 - acc: 0.8750 - val_loss: 0.2381 - val_acc: 0.8947
Epoch 34/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2458 - acc: 0.8957 - val_loss: 0.2347 - val_acc: 0.8972
Epoch 35/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2643 - acc: 0.8894 - val_loss: 0.2488 - val_acc: 0.8922
Epoch 36/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2426 - acc: 0.8869 - val_loss: 0.5529 - val_acc: 0.8521
Epoch 37/100
1592/1592 [==============================] - 246s 155ms/step - loss: 0.2667 - acc: 0.8769 - val_loss: 0.2174 - val_acc: 0.9248
Epoch 38/100
1592/1592 [==============================] - 248s 156ms/step - loss: 0.2950 - acc: 0.8693 - val_loss: 0.2781 - val_acc: 0.8922
Epoch 39/100
1592/1592 [==============================] - 246s 155ms/step - loss: 0.2410 - acc: 0.8945 - val_loss: 0.2123 - val_acc: 0.9148
Epoch 40/100
1592/1592 [==============================] - 248s 156ms/step - loss: 0.2352 - acc: 0.8957 - val_loss: 0.2122 - val_acc: 0.9223
Epoch 41/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2265 - acc: 0.9058 - val_loss: 0.1939 - val_acc: 0.9223
Epoch 42/100
1592/1592 [==============================] - 248s 156ms/step - loss: 0.2273 - acc: 0.9033 - val_loss: 0.2210 - val_acc: 0.9248
Epoch 43/100
1592/1592 [==============================] - 249s 156ms/step - loss: 0.2060 - acc: 0.9146 - val_loss: 0.1903 - val_acc: 0.9223
Epoch 44/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2133 - acc: 0.8970 - val_loss: 0.2516 - val_acc: 0.9148
Epoch 45/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2172 - acc: 0.9102 - val_loss: 0.3828 - val_acc: 0.8697
Epoch 46/100
1592/1592 [==============================] - 246s 155ms/step - loss: 0.1885 - acc: 0.9240 - val_loss: 0.2486 - val_acc: 0.9023
Epoch 47/100
1592/1592 [==============================] - 246s 155ms/step - loss: 0.2195 - acc: 0.9014 - val_loss: 0.2153 - val_acc: 0.9173
Epoch 48/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2061 - acc: 0.9102 - val_loss: 0.1922 - val_acc: 0.9373
Epoch 49/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2002 - acc: 0.9165 - val_loss: 0.1797 - val_acc: 0.9323
Epoch 50/100
1592/1592 [==============================] - 249s 156ms/step - loss: 0.1939 - acc: 0.9139 - val_loss: 0.2536 - val_acc: 0.9148
Epoch 51/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.2025 - acc: 0.9234 - val_loss: 0.2074 - val_acc: 0.9223
Epoch 52/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.1888 - acc: 0.9190 - val_loss: 0.2761 - val_acc: 0.8897
Epoch 53/100
1592/1592 [==============================] - 246s 155ms/step - loss: 0.1804 - acc: 0.9177 - val_loss: 0.3558 - val_acc: 0.8822
Epoch 54/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.1900 - acc: 0.9227 - val_loss: 0.4040 - val_acc: 0.7820
Epoch 55/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.1871 - acc: 0.9171 - val_loss: 0.2429 - val_acc: 0.9073
Epoch 56/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.1747 - acc: 0.9347 - val_loss: 0.2736 - val_acc: 0.9098
Epoch 57/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.1643 - acc: 0.9309 - val_loss: 0.1852 - val_acc: 0.9298
Epoch 58/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.1491 - acc: 0.9334 - val_loss: 0.3358 - val_acc: 0.8371
Epoch 59/100
1592/1592 [==============================] - 247s 155ms/step - loss: 0.1692 - acc: 0.9296 - val_loss: 0.1711 - val_acc: 0.9449
Epoch 60/100
1592/1592 [==============================] - 249s 156ms/step - loss: 0.1665 - acc: 0.9340 - val_loss: 0.1893 - val_acc: 0.9323
Epoch 61/100
1592/1592 [==============================] - 248s 156ms/step - loss: 0.1526 - acc: 0.9422 - val_loss: 0.7670 - val_acc: 0.8271
Epoch 62/100
1360/1592 [========================>.....] - ETA: 31s - loss: 0.1703 - acc: 0.9309
---------------------------------------------------------------------------
KeyboardInterrupt                         Traceback (most recent call last)
<ipython-input-5-e03e9a10aac4> in <module>()
      2 print("du", labels.shape)
      3 parallel_model.fit(x=features, y=labels, 
----> 4                    batch_size=16, epochs=100, verbose=1, validation_split=0.2)
      5 model.save_weights("diagnosis.h5")

~/anaconda3/envs/algo-work/lib/python3.5/site-packages/keras/engine/training.py in fit(self, x, y, batch_size, epochs, verbose, callbacks, validation_split, validation_data, shuffle, class_weight, sample_weight, initial_epoch, steps_per_epoch, validation_steps, **kwargs)
   1040                                         initial_epoch=initial_epoch,
   1041                                         steps_per_epoch=steps_per_epoch,
-> 1042                                         validation_steps=validation_steps)
   1043 
   1044     def evaluate(self, x=None, y=None,

~/anaconda3/envs/algo-work/lib/python3.5/site-packages/keras/engine/training_arrays.py in fit_loop(model, f, ins, out_labels, batch_size, epochs, verbose, callbacks, val_f, val_ins, shuffle, callback_metrics, initial_epoch, steps_per_epoch, validation_steps)
    197                     ins_batch[i] = ins_batch[i].toarray()
    198 
--> 199                 outs = f(ins_batch)
    200                 if not isinstance(outs, list):
    201                     outs = [outs]

~/anaconda3/envs/algo-work/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py in __call__(self, inputs)
   2659                 return self._legacy_call(inputs)
   2660 
-> 2661             return self._call(inputs)
   2662         else:
   2663             if py_any(is_tensor(x) for x in inputs):

~/anaconda3/envs/algo-work/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py in _call(self, inputs)
   2629                                 symbol_vals,
   2630                                 session)
-> 2631         fetched = self._callable_fn(*array_vals)
   2632         return fetched[:len(self.outputs)]
   2633 

~/anaconda3/envs/algo-work/lib/python3.5/site-packages/tensorflow/python/client/session.py in __call__(self, *args)
   1449         if self._session._created_with_new_api:
   1450           return tf_session.TF_SessionRunCallable(
-> 1451               self._session._session, self._handle, args, status, None)
   1452         else:
   1453           return tf_session.TF_DeprecatedSessionRunCallable(

KeyboardInterrupt: 


​


​

​


 ### use Adadelta
 (1991, 128, 128, 512)
du (1991, 2)
Train on 1592 samples, validate on 399 samples
Epoch 1/100
1592/1592 [==============================] - 258s 162ms/step - loss: 3.8041 - acc: 0.7575 - val_loss: 4.0396 - val_acc: 0.7494
Epoch 2/100
1592/1592 [==============================] - 245s 154ms/step - loss: 3.8270 - acc: 0.7626 - val_loss: 4.0396 - val_acc: 0.7494
Epoch 3/100
1592/1592 [==============================] - 246s 155ms/step - loss: 3.8270 - acc: 0.7626 - val_loss: 4.0396 - val_acc: 0.7494
Epoch 4/100
1592/1592 [==============================] - 247s 155ms/step - loss: 3.8270 - acc: 0.7626 - val_loss: 4.0396 - val_acc: 0.7494
Epoch 5/100
1592/1592 [==============================] - 245s 154ms/step - loss: 3.8270 - acc: 0.7626 - val_loss: 4.0396 - val_acc: 0.7494
Epoch 6/100
1592/1592 [==============================] - 245s 154ms/step - loss: 3.8270 - acc: 0.7626 - val_loss: 4.0396 - val_acc: 0.7494
Epoch 7/100
1592/1592 [==============================] - 246s 155ms/step - loss: 3.8270 - acc: 0.7626 - val_loss: 4.0396 - val_acc: 0.7494
Epoch 8/100
1592/1592 [==============================] - 246s 154ms/step - loss: 3.8270 - acc: 0.7626 - val_loss: 4.0396 - val_acc: 0.7494
Epoch 9/100
 992/1592 [=================>............] - ETA: 1:21 - loss: 3.7696 - acc: 0.7661
 